# Attraction Data Preprocessing
We crawled two websites, namely *Ctrip.com* and *Dianping.com*. \
The crawled content is the hot scores, number of comments and scores of Macau attractions on *Ctrip.com* and the number of comments and scores of Macau attractions on *Dianping.com*. They are stored in attraction_ctrip.xlsx and attraction_dianping.xlsx respectively.\
Because I had done the initial processing of the collected data manually, they became very neat and tidy.

## Prepare
The purpose of the first module is to prepare for subsequent data preprocessing, which includes importing relevant libraries, defining some reusable functions, etc. 

In [1]:
import pandas as pd
from tabulate import tabulate

def display_df_data(df, num_rows=3):
    """
    Display the first few rows of data in the dataframe.

    Parameters:
        df (dataframe): The dataframe which contains the data.
        num_rows (int): The number of rows to display initially. Default is 3.
    """
    # If the DataFrame has more rows than we want to display, insert an ellipsis
    if len(df) > num_rows + 1:
        ellipsis = pd.DataFrame([['...'] * len(df.columns)], columns=df.columns)
        df = pd.concat([df.head(num_rows), ellipsis, df.tail(1)], ignore_index=True)

    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))


ctrip_attraction_file_path = '../data/raw/attractions_information/attractions_information_ctrip.xlsx'
dianping_attraction_file_path = '../data/raw/attractions_information/attractions_information_dianping.xlsx'
attraction_file_path = '../data/processed/attractions_information.xlsx'

print("The initial data in attractions_information_ctrip.xlsx:")
display_df_data(pd.read_excel(ctrip_attraction_file_path))

print("\nThe initial data in attractions_information_dianping.xlsx:")
display_df_data(pd.read_excel(dianping_attraction_file_path))

The initial data in attractions_information_ctrip.xlsx:
+------------------+--------------------+-------------+---------+
| 标题             | hot_score_number   | recomment   | score   |
|------------------+--------------------+-------------+---------|
| 澳门科学馆       | 7.0                | (763条点评) | 4.8 分  |
| 澳门大赛车博物馆 | 5.8                | (126条点评) | 4.7 分  |
| 海事博物馆       | 3.9                | (137条点评) | 4.7 分  |
| ...              | ...                | ...         | ...     |
| 东方基金会会址   | 1.8                | (23条点评)  | 4 分    |
+------------------+--------------------+-------------+---------+

The initial data in attractions_information_dianping.xlsx:
+------------------+----------+----------+--------+
| 标题             | tag      | 评价数   | 评分   |
|------------------+----------+----------+--------|
| 圣若瑟修院藏珍馆 | 宗教     | 98       | 4.7    |
|                  |          | 条评价   |        |
| 女娲庙           | 宗教     | 97       | 4.1    |
|                  |          | 条评价   |       

## Merge two excel sheets
We use *outer joins* to merge the attraction data from the two websites, and the attractions are named connection keys. \
\
When using outer joins to merge these two scenic spot data sets, there will be many reasons why the matching is unsuccessful. Some columns of some rows will be filled with NaN, so subsequent manual adjustments are required. \
The following are the reasons I summarized for unsuccessful matching:
1. *Spelling differences*: There may be slight spelling differences in the names of attractions in the two datasets, such as differences in Cantonese and Chinese characters.
2. *Updates are out of sync*: The data updates on the two websites are out of sync, resulting in the name of the attraction on one website having been modified, while the name of the attraction on the other website has not been updated. Such as 民政总署大楼（现市政署大楼）.
3. *Different naming conventions*: For example, it is called "澳门塔" on Ctrip.com, but it is called "澳门旅游塔" on Dianping.com.

In [2]:
df1 = pd.read_excel(ctrip_attraction_file_path)
df2 = pd.read_excel(dianping_attraction_file_path)

df_attraction = pd.merge(df1, df2, on='标题', how='outer')

display_df_data(df_attraction)

+------------------+--------------------+-------------+---------+----------+----------+--------+
| 标题             | hot_score_number   | recomment   | score   | tag      | 评价数   | 评分   |
|------------------+--------------------+-------------+---------+----------+----------+--------|
| 澳门科学馆       | 7.0                | (763条点评) | 4.8 分  | 展览馆   | 1240     | 4.9    |
|                  |                    |             |         |          | 条评价   |        |
| 澳门大赛车博物馆 | 5.8                | (126条点评) | 4.7 分  | 展览馆   | 486      | 4.8    |
|                  |                    |             |         |          | 条评价   |        |
| 海事博物馆       | 3.9                | (137条点评) | 4.7 分  | 展览馆   | 300      | 4.8    |
|                  |                    |             |         |          | 条评价   |        |
| ...              | ...                | ...         | ...     | ...      | ...      | ...    |
| 友谊大桥         | nan                | nan         | nan     | 更多景点 | 100      | 4.7  

## Modify column name
Change *Chinese* or *inappropriate* column names to corresponding *English* column names.

In [3]:
new_column_names = {'标题': 'Attraction', 
                    'hot_score_number': 'ctrip_hot_score',
                    'recomment': 'ctrip_comment',
                    'score': 'ctrip_score',
                    '评价数': 'dianping_comment',
                    '评分': 'dianping_score'}
df_attraction = df_attraction.rename(columns=new_column_names)

display_df_data(df_attraction)

+------------------+-------------------+-----------------+---------------+----------+--------------------+------------------+
| Attraction       | ctrip_hot_score   | ctrip_comment   | ctrip_score   | tag      | dianping_comment   | dianping_score   |
|------------------+-------------------+-----------------+---------------+----------+--------------------+------------------|
| 澳门科学馆       | 7.0               | (763条点评)     | 4.8 分        | 展览馆   | 1240               | 4.9              |
|                  |                   |                 |               |          | 条评价             |                  |
| 澳门大赛车博物馆 | 5.8               | (126条点评)     | 4.7 分        | 展览馆   | 486                | 4.8              |
|                  |                   |                 |               |          | 条评价             |                  |
| 海事博物馆       | 3.9               | (137条点评)     | 4.7 分        | 展览馆   | 300                | 4.8              |
|                  |                 

## Delete unnecessary columns
Delete columns that are not useful for subsequent operations.

In [4]:
columns_to_drop = ["tag"]  # List of column names to delete
df_attraction.drop(columns=columns_to_drop, inplace=True)

display_df_data(df_attraction)

+------------------+-------------------+-----------------+---------------+--------------------+------------------+
| Attraction       | ctrip_hot_score   | ctrip_comment   | ctrip_score   | dianping_comment   | dianping_score   |
|------------------+-------------------+-----------------+---------------+--------------------+------------------|
| 澳门科学馆       | 7.0               | (763条点评)     | 4.8 分        | 1240               | 4.9              |
|                  |                   |                 |               | 条评价             |                  |
| 澳门大赛车博物馆 | 5.8               | (126条点评)     | 4.7 分        | 486                | 4.8              |
|                  |                   |                 |               | 条评价             |                  |
| 海事博物馆       | 3.9               | (137条点评)     | 4.7 分        | 300                | 4.8              |
|                  |                   |                 |               | 条评价             |                  |
| ..

## Remove unnecessary characters
We perform data processing operations on specific columns, including removing brackets and Chinese characters in columns. This cleans the data and saves the processed data in a normalized format for subsequent use.

In [5]:
columns = ['ctrip_hot_score', 'ctrip_comment', 'ctrip_score', 'dianping_comment', 'dianping_score']

for column in columns:
    # Check whether the column data type is string
    if pd.api.types.is_string_dtype(df_attraction[column]):
        df_attraction[column] = df_attraction[column].str.replace("[()]", '', regex=True) # remove ( and )
        df_attraction[column] = df_attraction[column].str.replace(r'[\u4e00-\u9fa5]+', '', regex=True) # remove chinese characters
    else:
        # If the column is not of string type, convert it to a string for operation
        df_attraction[column] = df_attraction[column].astype(str).str.replace("[()]", '', regex=True)
        df_attraction[column] = df_attraction[column].astype(str).str.replace(r'[\u4e00-\u9fa5]+', '', regex=True)

display_df_data(df_attraction)

df_attraction.to_excel(attraction_file_path, index=False)

+------------------+-------------------+-----------------+---------------+--------------------+------------------+
| Attraction       | ctrip_hot_score   | ctrip_comment   | ctrip_score   | dianping_comment   | dianping_score   |
|------------------+-------------------+-----------------+---------------+--------------------+------------------|
| 澳门科学馆       | 7.0               | 763             | 4.8           | 1240               | 4.9              |
| 澳门大赛车博物馆 | 5.8               | 126             | 4.7           | 486                | 4.8              |
| 海事博物馆       | 3.9               | 137             | 4.7           | 300                | 4.8              |
| ...              | ...               | ...             | ...           | ...                | ...              |
| 友谊大桥         | nan               | nan             | nan           | 100                | 4.7              |
+------------------+-------------------+-----------------+---------------+--------------------+-------